<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/hf_utilisation_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 101 kB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 29.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Tokenizer de Transformers, tout à savoir
 * pour l'instancier `tokenizer = AutoTokenizer.from_pretrained(checkpoint)`
 * pour padder, argument `padding = {longuest,max_length}` longuest pour la phrase la plus longue, max_length pour le maximum possible pour le modèle, i.e. 512 pour BERT
 * pour tronquer, argument `truncation=True` couplé avec l'argument `max_length=?` qui spécifie la taille maximale d'une phrase 
 * pour retourner des tensors au format approprié, argument `return_tensors={'pt','tf','np'}` pt pour pytorch et tensorflow pour tf
 * Attention ne pas oublier que le tokenizer est succeptible (selon le modèle) de rajouter des tokens spéciaux
 * Lorsque le tokenizer renvoie un dictionnaire avec mask de padding, on doit rajouter les deux étoiles `**` avant le dictionnaire quand on applique le modèle dessus

### Tokens Spéciaux

In [8]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."
# « J'ai attendu un cours de HuggingFace toute ma vie. »

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [9]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


## Format kwargs

In [10]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!",
]  # « J'ai attendu un cours de HuggingFace toute ma vie. », « Moi aussi ! »

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
print(tokens)


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [ ]:
output = model(**tokens)